## Import all what we need.

In [1]:
# encoding: utf-8
import os
import bz2
import csv
import glob
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from numba import njit
from io import StringIO
import matplotlib.pyplot as plt
import threading, multiprocessing

## The Input File Path and Parameters


<table><tr><td bgcolor=skyblue><font size=24> Could be changed ! </font></td></tr></table>

In [2]:
#########################################################
inp_filepath = '/home/jingxin/PyExoCross/input/NO_exomol.inp'
read_path = '/mnt/data/exomol/exomol3_data/'
save_path = '/home/jingxin/data/pyexocross/'
if os.path.exists(save_path):
    pass
else:
    os.makedirs(save_path, exist_ok=True)

ratio = 1.0
ratio_air = 0.7
ratio_self = 0.3
cutoff = 25.0
threshold = 1e-30
P = 1.0
#########################################################

## Report time

In [3]:
class Timer:    
    def start(self):
        self.start_CPU = time.process_time()
        self.start_sys = time.time()
        return self

    def end(self, *args):
        self.end_CPU = time.process_time()
        self.end_sys = time.time()
        self.interval_CPU = self.end_CPU - self.start_CPU
        self.interval_sys = self.end_sys - self.start_sys
        print('{:25s} : {}'.format('Running time on CPU', self.interval_CPU), 's')
        print('{:25s} : {}'.format('Running time on system', self.interval_sys), 's')

## Read Information from Input File

In [60]:
def inp_para(inp_filepath, N_A):
    
    inp_col_name = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10']
    inp_df = pd.read_csv(inp_filepath, sep='\\s+', names=inp_col_name, header=None)
    col0 = inp_df['c0']
    database = inp_df[col0.isin(['Database'])]['c1'].values[0]
    molecule = inp_df[col0.isin(['Molecule'])]['c1'].values[0]
    isotopologue = inp_df[col0.isin(['Isotopologue'])]['c1'].values
    dataset = inp_df[col0.isin(['Dataset'])]['c1'].values
    mol_iso_id = int(inp_df[col0.isin(['mol_iso_id'])]['c1'])
    T = int(inp_df[col0.isin(['Temperature'])]['c1'])
    min_wn = float(inp_df[col0.isin(['Range'])]['c1'])
    max_wn = float(inp_df[col0.isin(['Range'])]['c2'])
    N_point = int(inp_df[col0.isin(['Npoints'])]['c1'])
    abs_emi = inp_df[col0.isin(['AbsorptionOREmission'])]['c1'].values[0]
    profile = inp_df[col0.isin(['Profile'])]['c1'].values[0]
    wn_wl = inp_df[col0.isin(['Wavenumber(wn)/wavelength(wl)'])]['c1'].values[0]
    saveplots = inp_df[col0.isin(['SavePlots'])]['c1'].values[0]
    savexsecs = inp_df[col0.isin(['SaveXsecs'])]['c1'].values[0]

    molecule_id = int(mol_iso_id/10)
    isotopologue_id = mol_iso_id - molecule_id * 10
    isometa_url = 'https://hitran.org/docs/iso-meta/'
    iso_meta_table = pd.read_html(isometa_url)[molecule_id - 1]
    iso_meta_row = iso_meta_table[iso_meta_table['local ID'].isin([isotopologue_id])]
    abundance = float(iso_meta_row['Abundance'][0].replace('\xa0×\xa010','E'))
    
    if database == 'ExoMol':
        # Read ExoMol definition file (.def) to get the mass.
        deffile_path = (read_path+'/'+molecule+'/'+isotopologue+'/'+dataset+'/'+isotopologue+'__'+dataset+'.def')
        def_df = pd.read_csv(deffile_path[0],sep='\\s+',usecols=[0,1,2,3,4],names=['0','1','2','3','4'],header=None)
        mass_exomol = def_df[def_df['4'].isin(['mass'])]['1'].values[0]     # ExoMol mass (kg)
        mass = mass_exomol
    elif database == 'HITRAN':
        mass_hitran = float(iso_meta_row['Molar Mass /g·mol-1'])            # HITRAN molar mass (kg/mol)
        mass = mass_hitran / N_A
    else:
        print('Please add the name of the database ExoMol or HITRAN into the input file.')
    

    
    return database, molecule, isotopologue, dataset, T, min_wn, max_wn, N_point, abs_emi, profile, wn_wl, saveplots, savexsecs, molecule_id, isotopologue_id, abundance, mass


## Constants and Parameters

In [15]:
# Parameters for calculating.
import astropy.constants as ac
T_ref = 296.0                       # Reference temperature is 296 K.
P_ref = 1.0                         # Reference pressure is 1 atm.
N_A = ac.N_A.value                  # Avogadro number (1/mol)
h = ac.h.to('J s').value            # Planck's const (J s)
c = ac.c.to('cm/s').value           # Velocity of light (cm/s)
kB = ac.k_B.to('J/K').value         # Boltzmann's const (J/K)
R = ac.R.to('J / (K mol)').value    # Gas constant (J/(K mol))
c2 = h * c / kB                     # Second radiation constant (cm K)

(database, molecule, isotopologue, dataset, T, min_wn, max_wn, N_point, abs_emi, profile, wn_wl, 
saveplots, savexsecs, molecule_id, isotopologue_id, abundance, mass) = inp_para(inp_filepath, N_A)
c2_T = c2 / T                       # c2 / T (cm)
c2_T_ref = c2 / T_ref               # c2 / T_ref (cm)
pi_c_8 = 1 / (8 * np.pi * c)        # 8 * pi * c (s/cm)

# Read Input Files

Read the parameters of the linelist in ExoMol or HITRAN format text file. Return the dataframe of the data for the following calculations.

## Read ExoMol Database Files

### Read States File

In [16]:
def read_all_states(read_path):
    
    s_df = dict()
    states_df = pd.DataFrame()
    states_filenames = glob.glob((read_path + molecule + '/' + isotopologue + '/' + dataset 
                                  + '/' + isotopologue + '__' + dataset + '.states.bz2')[0])

    for states_filename in states_filenames:
        s_df[states_filename] = pd.read_csv(states_filename, compression='bz2', sep='\s+', header=None,
                                            chunksize=100_000_000, iterator=True, low_memory=False)
        for chunk in s_df[states_filename]:
            states_df = pd.concat([states_df, chunk])
            
    pd.set_option("display.max_columns",30)                           
    return(states_df)

In [17]:
#read_all_states(read_path)

### Read transitions File

In [18]:
def get_transfiles(read_path):
    # Get all the transitions files from the folder including the older version files which are named by vn(version number).
    trans_filepaths_all = glob.glob((read_path + molecule + '/' + isotopologue + '/' + dataset + '/' + '*trans.bz2')[0])
    num_transfiles_all = len(trans_filepaths_all)    # The number of all transitions files including the older version files.
    trans_filepaths = []    # The list of the lastest transitions files.
    for i in range(num_transfiles_all):
        split_version = trans_filepaths_all[i].split('__')[-1].split('.')[0].split('_')    # Split the filenames.
        num = len(split_version)
        # There are three format filenames.
        # The lastest transitions files named in two formats:
        # 1. Filenames are named with the name of isotopologue and dataset. 
        #    End with .trans.bz2.
        #    e.g. 14N-16O__XABC.trans.bz2'
        # 2. Filenames are named with the name of isotopologue and dataset. 
        #    Also have the range of wavenumbers xxxxx-yyyyy.
        #    End with .trans.bz2.
        #    e.g. 1H2-16O__POKAZATEL__00000-00100.trans.bz2
        # 3. The older version transitions files are named with vn(version number) based on the first format of the lastest files.
        #    e.g. 14N-16O__XABC_v2.trans.bz2
        # After split the filenames:
        # The first format filenames only leave the dataset name, e.g. XABC.
        # The second format filenames only leave the range of the wavenumber, e.g. 00000-00100.
        # The third format filenames leave two parts(dataset name and version number), e.g. XABC and v2.
        # This program only process the lastest data, so extract the filenames named by the first two format.
        if num == 1:        
            trans_filepaths.append(trans_filepaths_all[i])
    return(trans_filepaths)   

In [19]:
def read_all_trans(read_path):
    
    t_df = dict()
    trans_df = pd.DataFrame()
    trans_col_name = ['u', 'l', 'A', 'v']
    trans_filepaths = get_transfiles(read_path)
    #trans_filepaths = glob.glob((read_path + molecule + '/' + isotopologue + '/' + dataset + '/' + '*trans.bz2')[0])

    for trans_filename in tqdm(trans_filepaths):
        t_df[trans_filename] = pd.read_csv(trans_filename, compression='bz2', sep='\s+', header=None,
                                           names=trans_col_name, chunksize=100_000_000, iterator=True, low_memory=False)
        for chunk in t_df[trans_filename]:
            trans_df = pd.concat([trans_df,chunk])
            
    pd.set_option("display.max_columns",30)                           
    return(trans_df)
    

In [20]:
#read_all_trans(read_path)

### Read Partition Function File

In [21]:
def read_exomolweb_pf(T_ref, T):
    
    pf_url = ('http://www.exomol.com/db/' + molecule + '/' + isotopologue + '/' + dataset 
              + '/' + isotopologue + '__' + dataset + '.pf')[0]
    pf_content = requests.get(pf_url).text
    pf_col_name = ['T', 'Q']
    pf_df = pd.read_csv(StringIO(pf_content), sep='\\s+', names=pf_col_name, header=None)
    Q_ref = pf_df['Q'][T_ref-1]
    Q = pf_df['Q'][T-1]

    return(Q_ref, Q)

In [22]:
def read_exomol_pf(read_path, T_ref, T):
    
    pf_filename = (read_path + molecule + '/' + isotopologue + '/' + dataset 
                   + '/' + isotopologue + '__' + dataset + '.pf')[0]
    pf_col_name = ['T', 'Q']
    pf_df = pd.read_csv(pf_filename, sep='\\s+', names=pf_col_name, header=None)
    Q_ref = pf_df['Q'][T_ref-1]
    Q = pf_df['Q'][T-1]

    return(Q_ref, Q)

### Read Broadening File

In [23]:
def read_broad(read_path):
    
    broad_col_name = ['code', 'gamma_L', 'n_air', 'Jpp']
    broad_path = (read_path + molecule + '/' + isotopologue + '/' + isotopologue + '__air.broad')[0]
    broad_df = pd.read_csv(broad_path, sep='\s+', names=broad_col_name, header=None, engine='python')
    return(broad_df)

## Read HITRAN Database Files

### Read HITRAN Linelist File

In [24]:
def read_hitran_parfile (par_filepath, parfile_df):
    '''
    Read the parameters of the molecular absorption features
    of HITRAN2020 format text file.
    
    Parameters
    ----------
    par_filepath : str
        Input file path for reading.
    Return
    ------
    hitran_df : DataFrame
        The DataFrame of HITRAN data for the molecule.
    '''    
    par_filename = par_filepath.split('/')[3]
    
    if not os.path.exists:
        raise ImportError('The input file ' + par_filename + ' does not exist.')

    if (len(str(parfile_df[0][0])) < 160):
        raise ImportError('The file ' + par_filename + ' is not a HITRAN2016 format data file.')
    #hitran_column_name = ['M','I','v','S','Acoeff','gamma_air','gamma_self',
    #                     'Epp','n_air','delta_air','Vp','Vpp','Qp','Qpp',
    #                     'Ierr','Iref','flag','gp','gpp']

    hitran_df = pd.DataFrame()
    hitran_df['M'] = pd.to_numeric(parfile_df[0].map(lambda x: x[0:2]), errors='coerce').astype('int32')                 # Molecule identification number
    hitran_df['I'] = pd.to_numeric(parfile_df[0].map(lambda x: x[2:3]), errors='coerce').astype('int32')                 # Isotopologue number
    hitran_df['v'] = pd.to_numeric(parfile_df[0].map(lambda x: x[3:15]), errors='coerce').astype('float64')              # Transition wavenumber (in cm^{-1})
    hitran_df['S'] = pd.to_numeric(parfile_df[0].map(lambda x: x[15:25]), errors='coerce').astype('float64')             # Intensity (cm^{-1} / (molecule cm^{-2}))
    hitran_df['A'] = pd.to_numeric(parfile_df[0].map(lambda x: x[25:35]), errors='coerce').astype('float64')             # Einstein A-coefficient (in s^{-1})
    hitran_df['gamma_air'] = pd.to_numeric(parfile_df[0].map(lambda x: x[35:40]), errors='coerce').astype('float64')     # Air-broadened half-width at half maximum (HWHM) coefficient (cm^{-1} atm^{-1})
    hitran_df['gamma_self'] = pd.to_numeric(parfile_df[0].map(lambda x: x[40:45]), errors='coerce').astype('float64')    # Self-broadened half-width at half maximum (HWHM) coefficient (cm^{-1} atm^{-1})
    hitran_df['Epp'] = pd.to_numeric(parfile_df[0].map(lambda x: x[45:55]), errors='coerce').astype('float64')           # Lower state energy (cm^{-1})
    hitran_df['n_air'] = pd.to_numeric(parfile_df[0].map(lambda x: x[55:59]), errors='coerce').astype('float64')         # Temperature-dependent exponent for gamma_air
    #hitran_df['delta_air'] = pd.to_numeric(parfile_df[0].map(lambda x: x[59:67]), errors='coerce').astype('float64')     # Air pressure_include line shift (cm^{-1} atm^{-1})
    #hitran_df['gp'] = pd.to_numeric(parfile_df[0].map(lambda x: x[146:153]), errors='coerce').astype('float64')        # Statistical weight of the upper state
    #hitran_df['gpp'] = pd.to_numeric(parfile_df[0].map(lambda x: x[153:160]), errors='coerce').astype('float64')        # Statistical weight of the lower state

    hitran_df = hitran_df[hitran_df['M'].isin([molecule_id])]
    hitran_df = hitran_df[hitran_df['I'].isin([isotopologue_id])]
    hitran_df = hitran_df[hitran_df['v'].between(min_wn, max_wn)]
    hitran_df = hitran_df[hitran_df['S'] > threshold]
    
    return hitran_df


### Read Partition Function File

In [25]:
def read_hitran_pf(T):
    
    isometa_url = 'https://hitran.org/docs/iso-meta/'
    iso_meta_table = pd.read_html(isometa_url)[molecule_id - 1]
    iso_meta_row = iso_meta_table[iso_meta_table['local ID'].isin([isotopologue_id])]
    Q_ref = float(iso_meta_row.loc[0][6].replace('\xa0×\xa010','E'))
    Q_url = 'https://hitran.org/data/Q/' + iso_meta_row.loc[0][7]
    Q_content = requests.get(Q_url).text
    Q_col_name = ['T', 'Q']
    Q_df = pd.read_csv(StringIO(Q_content), sep='\\s+', names=Q_col_name, header=None)
    Q = Q_df['Q'][T - 1]   
    return(Q_ref, Q)

# Calculate Parition Function


In [26]:
def calculate_partition(E, g, T):
    partition_func = np.sum(g * np.exp(-c2 * E / T))
    return(partition_func)

In [27]:
# Partition function
def exomol_partition_func(read_path, Ntemp, Tmax):
    
    states_df = read_all_states(read_path)
    E = states_df[1]
    g = states_df[2]
    pf_list = []

    for T in tqdm(range(Ntemp, Tmax+1, Ntemp)):
        partition_func = calculate_partition(E, g, T)
        pf_list.append(str(format(T, '8.1f') + ' ' + format(partition_func, '15.4f')))
        
    pf_folder = save_path + '/partition/'
    if os.path.exists(pf_folder):
        pass
    else:
        os.makedirs(pf_folder, exist_ok=True)
    pf_path = (pf_folder + isotopologue + '__' + dataset + '.pf')[0]
 
    with open(pf_path, 'w') as pf_file:
        for pf in pf_list:
            pf_file.write(pf + '\n')
        pf_file.close()

    print('Partition function has been saved!')  


In [28]:
Ntemp = 1
Tmax = int(5000.00)
t = Timer()
t.start()
exomol_partition_func(read_path, Ntemp, Tmax)
t.end()


100%|██████████| 5000/5000 [00:03<00:00, 1378.60it/s]

Partition function has been saved!
Running time on CPU       : 3.786006424 s
Running time on system    : 3.7964038848876953 s


# Calculate Cooling Function

In [29]:
def all_linelist(read_path):
    states_df = read_all_states(read_path)
    trans_df = read_all_trans(read_path)
    
    id_u = trans_df['u'].values
    states_df.set_index([0], inplace=True, drop=False)
    states_u_df = states_df.loc[id_u]
    linelist_df = pd.DataFrame()
    linelist_df['u'] = trans_df['u']
    linelist_df['l'] = trans_df['l']
    linelist_df['A'] = trans_df['A']
    linelist_df['v'] = trans_df['v']
    linelist_df['Ep'] = states_u_df[1].values
    linelist_df['gp'] = states_u_df[2].values
    
    return linelist_df

In [30]:
#all_linelist_df = all_linelist(read_path)
#all_linelist_df

In [31]:
@njit(parallel=True, fastmath=True)
def calculate_cooling(A, v, Ep, gp, T, Q):
    cooling_func = np.sum(A * h * c * v * gp * np.exp(-c2 * Ep / T)) / (4 * np.pi * Q) * 1e7
    return(cooling_func)

In [34]:
# Cooling function
def exomol_cooling_func(read_path, Ntemp, Tmax):
    
    linelist_df = all_linelist(read_path)
    A = linelist_df['A'].values
    v = linelist_df['v'].values
    Ep = linelist_df['Ep'].values      # Lower state energy
    gp = linelist_df['gp'].values      # Total degeneracy of lower state 
    cf_list = []
    
    for T in tqdm(range(Ntemp, Tmax+1, Ntemp)):
        Q_ref, Q = read_exomol_pf(read_path, T_ref, T)
        cooling_func = calculate_cooling(A, v, Ep, gp, T, Q)
        cf_list.append(str(format(T, '8.1f') + ' ' + format(cooling_func, '20.8E')))

    cf_folder = save_path + '/cooling/'
    if os.path.exists(cf_folder):
        pass
    else:
        os.makedirs(cf_folder, exist_ok=True)  
    cf_path = (cf_folder + isotopologue + '__' + dataset + '.cooling')[0]
    
    with open(cf_path, 'w') as cf_file:
        for cf in cf_list:
            cf_file.write(cf + '\n')
        cf_file.close()

    print('Cooling function has been saved!')   
    


In [35]:
Ntemp = 1
Tmax = int(500.00)
t = Timer()
t.start()
exomol_cooling_func(read_path, Ntemp, Tmax)
t.end()

100%|██████████| 500/500 [00:03<00:00, 152.21it/s]

Cooling function has been saved!
Running time on CPU       : 521.90417973 s
Running time on system    : 16.756798028945923 s


# Lifetime

In [36]:
# Lifetime
def exomol_lifetime(read_path):
    
    states_df = read_all_states(read_path)
    trans_df = read_all_trans(read_path)
    sum_A = trans_df.groupby('u')['A'].sum()
    lifetime = 1 / sum_A
    lt_df = lifetime.map('{: >12.4E}'.format).reset_index()
    states_trans_df = pd.concat([states_df[0], lt_df['u']])
    states_trans_df = states_trans_df.drop_duplicates(keep=False)
    add_u = pd.DataFrame()
    add_u['u'] = states_trans_df
    add_u['A'] = '  Inf       '
    lifetime_df = pd.concat([lt_df, add_u], ignore_index=True)
    lifetime_df.sort_values('u',inplace=True)
    
    lf_folder = save_path + '/lifetime/'
    if os.path.exists(lf_folder):
        pass
    else:
        os.makedirs(lf_folder, exist_ok=True)  
    lf_path = (lf_folder + isotopologue + '__' + dataset + '.life')[0]
    lifetime_df['A'].to_csv(lf_path, header=None, index=False, chunksize = 1000)

    print('lifetime has been saved!')   
    return(lifetime_df)

In [37]:
exomol_lifetime(read_path)

100%|██████████| 1/1 [00:11<00:00, 11.02s/it]

lifetime has been saved!


,u,A
30810,1,Inf
0,2,8.3086E-02
1,3,4.2467E-02
2,4,2.8936E-02
3,5,2.2179E-02
...,...,...
30805,30807,2.2635E+00
30806,30808,2.2595E+00
30807,30809,2.1400E+00
30808,30810,2.2604E+00


# Specific Heat

In [38]:
@njit(parallel=True, fastmath=True)
def calculate_specific_heat(E, g, T):
    pf = np.sum(g * np.exp(-c2 * E / T))  
    pfp = np.sum(g * np.exp(-c2 * E / T) * (c2 * E / T))
    pfpp = np.sum(g * np.exp(-c2 * E / T) * (c2 * E / T) ** 2)
    specificheat_func = R * (pfpp / pf - (pfp / pf) ** 2) + 2.5 * R
    return(specificheat_func)

In [39]:
# Specific heat
def exomol_specificheat_func(read_path, Ntemp, Tmax):

    states_df = read_all_states(read_path)
    E = states_df[1].values
    g = states_df[2].values
    cp_list = []
    
    for T in tqdm(range(200, Tmax+1, Ntemp)):
        specificheat_func = calculate_specific_heat(E, g, T)
        cp_list.append(str(format(T, '8.1f') + ' ' + format(specificheat_func, '15.4f')))
        
    cp_folder = save_path + '/specific_heat/'
    if os.path.exists(cp_folder):
        pass
    else:
        os.makedirs(cp_folder, exist_ok=True)  
    cp_path = (cp_folder + isotopologue + '__' + dataset + '.cp')[0]
 
    with open(cp_path, 'w') as cp_file:
        for cp in cp_list:
            cp_file.write(cp + '\n')
        cp_file.close()

    print('Specific heat has been saved!')  


In [40]:
Ntemp = 1
Tmax = int(5000.0)
t = Timer()
t.start()
exomol_specificheat_func(read_path, Ntemp, Tmax)
t.end()


100%|██████████| 4801/4801 [00:01<00:00, 2645.35it/s]

Specific heat has been saved!
Running time on CPU       : 199.8381291620001 s
Running time on system    : 1.977797031402588 s


## Line Profile

In [ ]:
@njit(parallel=True, fastmath=True)
def Doppler_HWHM (v, mass, T):
    '''Return the Doppler half-width at half-maximum (HWHM) -- alpha.'''
    alpha = np.sqrt(2 * kB * T * np.log(2) / mass) * v / c
    return alpha

In [ ]:
@njit(parallel=True, fastmath=True)
def Lorentzian_HWHM (T, P, n_air, gamma_L):
    '''Return the Lorentzian half-width at half-maximum (HWHM) -- gamma.'''
    gamma = gamma_L * ratio * (T_ref / T)**n_air * (P / P_ref)
    
    return gamma

In [ ]:
@njit(parallel=True, fastmath=True)
def Gaussian_FWHM (alpha):
    f_G = 2 * alpha * np.sqrt(2 * np.log(2))
    return(f_G)

In [ ]:
@njit(parallel=True, fastmath=True)
def Lorentzian_FWHM (gamma):
    f_L = 2 * gamma
    return(f_L)

In [ ]:
@njit(parallel=True, fastmath=True)
def Gaussian_profile(dv, alpha):
    '''Return Gaussian line profile at dv with HWHM alpha.'''
    alpha = alpha * 2
    Gaussian_profile = np.sqrt(np.log(2) / np.pi) / alpha * np.exp(-np.log(2) * (dv / alpha)**2)
    return Gaussian_profile

In [ ]:
@njit(parallel=True, fastmath=True)
def Lorentzian_profile(dv, gamma):
    '''Return Lorentzian line profile at dv with HWHM gamma.'''
    #gamma = gamma * 2
    Lorentzian_profile = gamma / np.pi / (dv**2 + gamma**2)
    return Lorentzian_profile


In [ ]:
def scipy_Voigt_profile(dv, alpha, gamma):
    '''Return the Voigt line profile with Lorentzian component HWHM gamma and Gaussian component HWHM alpha.'''
    from scipy.special import voigt_profile
    return voigt_profile(dv, alpha, gamma)

In [ ]:
def scipy_wofz_Voigt_profile(dv, alpha, gamma):
    '''Return the Voigt line profile with Lorentzian component HWHM gamma and Gaussian component HWHM alpha.'''
    from scipy.special import wofz
    sigma = alpha / np.sqrt(2 * np.log(2))
    scipy_wofz_Voigt_profile = np.real(wofz((dv + 1j*gamma)/sigma/np.sqrt(2))) / sigma / np.sqrt(2*np.pi)
    return scipy_wofz_Voigt_profile

In [ ]:
@njit(parallel=True, fastmath=True)
def width_Voigt_profile(f_G, f_L):
    '''
    Return the full width at half maximum (FWHM) of the Voigt profile which can
    be found from the widths of the associated Gaussian and Lorentzian widths.

    '''
    width_Voigt_profile = 0.5346 * f_L + np.sqrt(0.2166 * f_L**2 + f_G**2)
    return width_Voigt_profile

In [ ]:
@njit(parallel=True, fastmath=True)
def pseudo_Voigt_profile(f_G, f_L):
    '''Return the Voigt line profile with Lorentzian component HWHM gamma and Gaussian component HWHM alpha.'''
    pseudo_Voigt_profile = (f_G**5+2.69269*f_G**4*f_L+2.42843*f_G**3*f_L**2+4.47163*f_G**2*f_L**3+0.07842*f_G*f_L**4+f_L**5)**0.2
    return pseudo_Voigt_profile

In [ ]:
@njit(parallel=True, fastmath=True)
def pseudo_Voigt_FWHM_profile(dv, alpha, gamma):
    '''
    Return the Voigt line profile with Lorentzian component HWHM gamma and Gaussian component HWHM alpha.
    f is the total FWHM parameter.
    n is a function of Lorentz (f_L), Gaussian (f_G) and total (f) full width at half maximum (FWHM) parameters.
    
    '''
    f_G = 2 * alpha * np.sqrt(2 * np.log(2))
    f_L = 2 * gamma
    f = (f_G**5 + 2.69269*f_G**4*f_L + 2.42843*f_G**3*f_L**2 + 4.47163*f_G**2*f_L**3 + 0.07842*f_G*f_L**4 + f_L**5)**0.2
    n = 1.36603*(f_L/f) - 0.47719*(f_L/f)**2 + 0.11116*(f_L/f)**3
    pseudo_fwhm_Voigt_profile = n*Lorentzian_profile(dv,f/2) + (1-n)*Gaussian_profile(dv,f/2)
    return pseudo_fwhm_Voigt_profile

In [ ]:
@njit(parallel=True, fastmath=True)
def pseudo_Voigt_FWHM_profile(dv, f_G, f_L):
    '''
    Return the Voigt line profile with Lorentzian component HWHM gamma and Gaussian component HWHM alpha.
    f is the total FWHM parameter.
    n is a function of Lorentz (f_L), Gaussian (f_G) and total (f) full width at half maximum (FWHM) parameters.
    
    '''
    f = (f_G**5 + 2.69269*f_G**4*f_L + 2.42843*f_G**3*f_L**2 + 4.47163*f_G**2*f_L**3 + 0.07842*f_G*f_L**4 + f_L**5)**0.2
    n = 1.36603*(f_L/f) - 0.47719*(f_L/f)**2 + 0.11116*(f_L/f)**3
    pseudo_fwhm_Voigt_profile = n*Lorentzian_profile(dv,f/2) + (1-n)*Gaussian_profile(dv,f/2)
    return pseudo_fwhm_Voigt_profile

## Calculate Cross Sections

In [ ]:
@njit(parallel=True, fastmath=True)
def cal_abscoefs(v, gp, A, Epp, Q, abundance):
    abscoef = gp * A * np.exp(- c2_T * Epp) * (1 - np.exp(- c2_T * v)) * pi_c_8 / (v ** 2) / Q * abundance   
    return (abscoef)

In [ ]:
@njit(parallel=True, fastmath=True)
def cal_emicoefs(v, gp, A, Ep, Q, abundance):
    emicoef = gp * A * v * np.exp(- c2_T * Ep) / (4 * np.pi * Q) * abundance   
    return (emicoef)

In [ ]:
def exomol_cross_section_Gaussian(wn_grid, v, alpha, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with Gaussian profile.
    
    '''
    xsec_g = np.zeros_like(wn_grid)
    _xsec_g = np.zeros_like(wn_grid)
    
    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = np.array(dv[filter_cutoff])
        _alpha = np.array(alpha[filter_cutoff])
        _coef = coef[filter_cutoff]
        _xsec_g[i] = np.sum(_coef * Gaussian_profile(_dv, _alpha))        

    xsec_g[0:N_point] += _xsec_g
    return (xsec_g)

In [ ]:
def exomol_cross_section_Lorentzian(wn_grid, v, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with Lorentzian profile.
    
    '''
    xsec_l = np.zeros_like(wn_grid)
    _xsec_l = np.zeros_like(wn_grid)
    
    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = np.array(dv[filter_cutoff])
        _gamma = np.array(gamma[filter_cutoff])
        _coef = coef[filter_cutoff]
        _xsec_l[i] = np.sum(_coef * Lorentzian_profile(_dv, _gamma))        

    xsec_l[0:N_point] += _xsec_l
    return (xsec_l)

In [ ]:
def exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with scipy Voigt profile.
    
    '''
    xsec_sv = np.zeros_like(wn_grid)
    _xsec_sv = np.zeros_like(wn_grid)

    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = dv[filter_cutoff]
        _alpha = alpha[filter_cutoff]
        _gamma = gamma[filter_cutoff]
        _coef = coef[filter_cutoff]
        _xsec_sv[i] = np.sum(_coef * scipy_Voigt_profile(_dv, _alpha, _gamma))

    xsec_sv[0:N_point] += _xsec_sv
    return (xsec_sv)

In [ ]:
def exomol_cross_section_scipy_wofz_Voigt(wn_grid, v, alpha, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with wofz Voigt profile.

    '''
    xsec_swv = np.zeros_like(wn_grid)
    _xsec_swv = np.zeros_like(wn_grid)

    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = dv[filter_cutoff]
        _alpha = alpha[filter_cutoff]
        _gamma = gamma[filter_cutoff]
        _coef = coef[filter_cutoff]
        _xsec_swv[i] = np.sum(_coef * scipy_wofz_Voigt_profile(_dv, _alpha, _gamma))

    xsec_swv[0:N_point] += _xsec_swv
    return (xsec_swv)

In [ ]:
def exomol_cross_section_width_Voigt(wn_grid, v, alpha, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with width Voigt profile.

    '''
    xsec_wv = np.zeros_like(wn_grid)
    _xsec_wv = np.zeros_like(wn_grid)

    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = dv[filter_cutoff]
        _alpha = alpha[filter_cutoff]
        _gamma = gamma[filter_cutoff]
        f_G = Gaussian_profile(_dv, _alpha)
        f_L = Lorentzian_profile(_dv, _gamma)
        _coef = coef[filter_cutoff]
        _xsec_wv[i] = np.sum(_coef * width_Voigt_profile(f_G, f_L))

    xsec_wv[0:N_point] += _xsec_wv
    return (xsec_wv)

In [ ]:
def exomol_cross_section_pseudo_Voigt(wn_grid, v, alpha, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with width Voigt profile.

    '''
    xsec_pv = np.zeros_like(wn_grid)
    _xsec_pv = np.zeros_like(wn_grid)

    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = dv[filter_cutoff]
        _alpha = alpha[filter_cutoff]
        _gamma = gamma[filter_cutoff]
        f_G = Gaussian_profile(_dv, _alpha)
        f_L = Lorentzian_profile(_dv, _gamma)
        _coef = coef[filter_cutoff]
        _xsec_pv[i] = np.sum(_coef * pseudo_Voigt_profile(f_G, f_L))

    xsec_pv[0:N_point] += _xsec_pv
    return (xsec_pv)

In [ ]:
def exomol_cross_section_pseudo_FWHM_Voigt(wn_grid, v, alpha, gamma, coef):
    '''
    Read ExoMol .states, .trans, .pf and .broad files as the input files.
    Return the wavennumbers and cross sections with width Voigt profile.

    '''
    xsec_pfv = np.zeros_like(wn_grid)
    _xsec_pfv = np.zeros_like(wn_grid)

    for i in range(N_point):
        dv = wn_grid[i] - v
        filter_cutoff = np.abs(dv) <=cutoff
        _dv = dv[filter_cutoff]
        _alpha = alpha[filter_cutoff]
        _gamma = gamma[filter_cutoff]
        f_G = Gaussian_FWHM(_alpha)
        f_L = Lorentzian_FWHM(_gamma)
        _coef = coef[filter_cutoff]
        _xsec_pfv[i] = np.sum(_coef * pseudo_Voigt_FWHM_profile(_dv, f_G, f_L))

    xsec_pfv[0:N_point] += _xsec_pfv
    return (xsec_pfv)

## Plot and Save Results

In [ ]:
def plot_xsec(wn, xsec, database, profile):
    print('{:25s} : {}'.format('Air broading is', ratio_air))
    print('{:25s} : {}'.format('Self broading is', ratio_self))
    print('{:25s} : {:<6}'.format('Temperature selected', T), 'K')
    print('{:25s} : {:<6}'.format('Pressure selected', P), 'bar')
    if (wn_wl == 'wn'):
        print('{:25s} : {:<6}'.format('Cutoff is', cutoff), 'cm-1')
        print('{:25s} : {:<6}'.format('Threshold is', threshold), 'cm-1/(molecule cm-2)')
        print('{:25s} : {} {} {} {}'.format('Wavenumber range selected', min_wn, 'cm-1 -', max_wn, 'cm-1'))
        
        # Plot cross sections and save it as .png.
        plt.plot(wn, xsec, label='T = '+str(T)+' K, '+profile)
        plt.title(database+' '+molecule+' '+abs_emi+' Cross-Section with '+ profile) 
        plt.xlabel('Wavenumber, cm$^{-1}$')
        plt.ylabel('Cross-section, cm$^{2}$/molecule')
        plt.legend()
        plots_foldername = saveplots+molecule+'/'
        if os.path.exists(plots_foldername):
            pass
        else:
            os.makedirs(plots_foldername, exist_ok=True)    
        plt.savefig(plots_foldername+molecule+'__T'+str(T)+'__'+str(min_wn)+'-'+str(max_wn)+'cm-1__'
                    +database+'__'+abs_emi+'__'+profile+'.png', dpi=500)
        plt.show()
        print('Cross sections plot saved.')

        # Save cross sections into .xsec file.
        xsec_df = pd.DataFrame()
        xsec_df['wavenumber'] = wn
        xsec_df['cross-section'] = xsec
        xsecs_foldername = savexsecs+molecule+'/'
        if os.path.exists(xsecs_foldername):
            pass
        else:
            os.makedirs(xsecs_foldername, exist_ok=True)
        xsec_filename = (xsecs_foldername+molecule+'__T'+str(T)+'__'+str(min_wn)+'-'+str(max_wn)+'cm-1__'
                         +database+'__'+abs_emi+'__'+profile+'.xsec')
        np.savetxt(xsec_filename, np.vstack((wn,xsec)).T)
        print('Cross sections file saved.')
        
    elif wn_wl == 'wl':
        wl = 10**7 / wn
        min_wl = 10**7 / min_wn
        max_wl = 10**7 / max_wn
        print('{:25s} : {:<6}'.format('Cutoff is', 10**7 / cutoff), 'nm')
        print('{:25s} : {:<6}'.format('Threshold is', 10**7 / threshold), 'nm/(molecule nm2)')
        print('{:25s} : {} {} {} {}'.format('Wavelength range selected', min_wl, 'nm -', max_wl, 'nm'))

        # Plot cross sections and save it as .png.
        plt.plot(wl, xsec, label='T = '+str(T)+' K, '+profile)
        plt.title(database+' '+molecule+' '+abs_emi+' Cross-Section with '+ profile) 
        plt.xlabel('Wavelength, nm')
        plt.ylabel('Cross-section, nm$^{-2}$/molecule')
        plt.legend()
        plots_foldername = saveplots+molecule+'/'
        if os.path.exists(plots_foldername):
            pass
        else:
            os.makedirs(plots_foldername, exist_ok=True)    
        plt.savefig(plots_foldername+molecule+'__T'+str(T)+'__'+str(min_wl)+'-'+str(max_wl)+'nm__'
                    +database+'__'+abs_emi+'__'+profile+'.png', dpi=500)
        plt.show()
        print('Cross sections plot saved.')
        
        # Save cross sections into .xsec file.
        xsec_df = pd.DataFrame()
        xsec_df['wavelength'] = wl
        xsec_df['cross-section'] = xsec
        xsecs_foldername = savexsecs+molecule+'/'
        if os.path.exists(xsecs_foldername):
            pass
        else:
            os.makedirs(xsecs_foldername, exist_ok=True)
        xsec_filename = (xsecs_foldername+molecule+'__T'+str(T)+'__'+str(min_wl)+'-'+str(max_wl)+'nm__'
                         +database+'__'+abs_emi+'__'+profile+'.xsec')
        np.savetxt(xsec_filename, np.vstack((wl,xsec)).T)
        print('Cross sections file saved.')

    else:
        print('Please type in correct format: wn or wl.')

## Get Results

In [ ]:
def part_linelist(read_path, trans_df):
    states_df = read_all_states(read_path)
    Q_ref, Q = read_exomol_pf(read_path, T_ref, T)
    broad_df = read_broad(read_path)
    id_u = trans_df['u'].values
    id_l = trans_df['l'].values
    states_df.set_index([0], inplace=True, drop=False)
    states_u_df = states_df.loc[id_u]
    states_l_df = states_df.loc[id_l]
    A = trans_df['A'].values
    v = trans_df['v'].values
    Ep = states_u_df[1].values
    Epp = states_l_df[1].values
    gp = states_u_df[2].values
    linelist_df = pd.DataFrame()
    max_broad_J = max(broad_df['Jpp'])
    linelist_df['Jpp'] = states_l_df[3].values
    linelist_df['Jpp'][linelist_df.Jpp > max_broad_J] = max_broad_J
    Jpp = linelist_df['Jpp'].values
    id_broad = linelist_df['Jpp'] - 0.5
    gamma_L = broad_df['gamma_L'][id_broad].values
    n_air = broad_df['n_air'][id_broad].values
    wn_grid = np.linspace(min_wn, max_wn, N_point)
    
    return (wn_grid, Q, A, v, Ep, Epp, gp, Jpp, gamma_L, n_air)

In [ ]:
def hitran_linelist(hitran_df):
    '''
    Read HITRAN .par file as the input file.
    Return the data for calculating wavennumbers and cross sections with line profiles.
    
    '''
    wn_grid = np.linspace(min_wn, max_wn, N_point)
    v = hitran_df['v'].values
    S = hitran_df['S'].values
    A = hitran_df['A'].values
    Epp = hitran_df['Epp'].values
    n_air = hitran_df['n_air'].values
    gamma_air = hitran_df['gamma_air'].values
    gamma_self = hitran_df['gamma_self'].values
    
    
    return (wn_grid, v, S, A, Epp, n_air, gamma_air, gamma_self)

In [ ]:
def get_results(df):
    if database == 'ExoMol':
        #states_df = read_all_states(read_path)
        #Q_ref, Q = read_pf(read_path, T_ref, T)
        #broad_df = read_broad(read_path)
        wn_grid, Q, A, v, Ep, Epp, gp, Jpp, gamma_L, n_air = part_linelist(read_path, df)

    elif database == 'HITRAN':
        hitran_df = read_hitran_parfile (par_filepath, df)
        wn_grid, v, S, A, Epp, n_air, gamma_air, gamma_self = hitran_linelist(hitran_df)
        
    else:
        print('Please add the name of the database ExoMol or HITRAN into the input file.')
    
    if abs_emi == 'Absorption':
        coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
    else:
        coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
    
    if profile == 'Gaussian':
        alpha = Doppler_HWHM (v, mass, T)
        xsec_g = exomol_cross_section_Gaussian(wn_grid, v, alpha, coef)
        plot_xsec(wn_grid, xsec_g, database, profile)
    elif profile == 'Lorentzian':
        gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
        xsec_l = exomol_cross_section_Lorentzian(wn_grid, v, gamma, coef)
        plot_xsec(wn_grid, xsec_l, database, profile)
    else:
        alpha = Doppler_HWHM (v, mass, T)
        gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
    if profile == 'SciPyVoigt':
        xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
        plot_xsec(wn_grid, xsec_sv, database, profile)
    elif profile == 'SciPyWofzVoigt':
        xsec_swv = exomol_cross_section_scipy_wofz_Voigt(wn_grid, v, alpha, gamma, coef)
        plot_xsec(wn_grid, xsec_swv, database, profile)
    elif profile == 'WidthVoigt':
        xsec_wv = exomol_cross_section_width_Voigt(wn_grid, v, alpha, gamma, coef)
        plot_xsec(wn_grid, xsec_wv, database, profile)
    elif profile == 'PseudoVoigt':
        xsec_pv = exomol_cross_section_pseudo_Voigt(wn_grid, v, alpha, gamma, coef)        
        plot_xsec(wn_grid, xsec_pv, database, profile)
    elif profile == 'PseudoFWHMVoigt':
        xsec_pfv = exomol_cross_section_pseudo_FWHM_Voigt(wn_grid, v, alpha, gamma, coef)
        plot_xsec(wn_grid, xsec_pfv, database, profile)
    else:
        print('Please choose line profile from the list.')
    pass

In [ ]:
def get_exomol_exocross(read_path):

    # Initialise the iterator object.
    trans_col_name = ['u', 'l', 'A', 'v']
    #trans_filepaths = get_transfiles(read_path)
    trans_filepaths = glob.glob((read_path + molecule + '/' + isotopologue + '/' + dataset + '/' + '*trans.bz2')[0])
    if len(trans_filepaths) == 1:
        transfile_df = pd.read_csv(trans_filepaths[0], compression='bz2', sep='\s+', header=None, 
                                names=trans_col_name, chunksize=100_000_000, encoding='utf-8')

    else:
        for trans_filename in tqdm(trans_filepaths):
            lower = int(trans_filename.split('__')[2].split('.')[0].split('-')[0])
            upper = int(trans_filename.split('__')[2].split('.')[0].split('-')[1]) 
            if (lower <= int(min_wn) <= upper):
                lowerfile = pd.read_csv(trans_filename, compression='bz2', sep='\s+', header=None, 
                                        names=trans_col_name, chunksize=100_000_000, encoding='utf-8')       
                #for lower_chunk in lowerfile:
                #    lower_chunk = lower_chunk[lower_chunk['v'].between(min_wn - cutoff, max_wn + cutoff)]     

            elif (lower <= int(max_wn) <= upper):
                upperfile = pd.read_csv(trans_filename, compression='bz2', sep='\s+', header=None, 
                                        names=trans_col_name, chunksize=100_000_000, encoding='utf-8')       
                #for upper_chunk in upperfile:
                #    upper_chunk = upper_chunk[upper_chunk['v'].between(min_wn - cutoff, max_wn + cutoff)]     
        
            elif (lower >= int(min_wn) and upper <= int(max_wn)):
                insiderfile = pd.read_csv(trans_filename, compression='bz2', sep='\s+', header=None, 
                                        names=trans_col_name, chunksize=100_000_000, encoding='utf-8')     
            transfile_df = lowerfile.append(insiderfile).append(upperfile)         
            
    # Depends on how many cores you want to utilise.
    # Reserve 64 cores for our script.
    max_processors = 4       
    pool = multiprocessing.Pool(processes=max_processors)
    f_list = []
    for trans_df in transfile_df:
        trans_df = trans_df[trans_df['v'].between(min_wn - cutoff, max_wn + cutoff)] 
        f = pool.apply_async(get_results, [trans_df])
        if len(pool._cache) > max_processors * 200:
            f.wait()
        f_list.append(f)
        if len(f_list) >= max_processors:
            for f in f_list:
                f.get()
                del f_list[:]
    pool.close()
    print('Finished!')

In [ ]:
if database == 'ExoMol':
    get_exomol_exocross(read_path)
elif database == 'HITRAN':
    wn_grid, v, S, A, Epp, n_air, gamma_air, gamma_self = hitran_linelist(df)
else:
    raise Exception('Please add the name of the database ExoMol or HITRAN into the input file.')

# Test

## Plot At The Same T With Different Line Profile

In [ ]:
T=500
wn_grid, Q, A, v, Ep, Epp, gp, Jpp, gamma_L, n_air = part_linelist(read_path, trans_df)
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)

In [ ]:
xsec_g = exomol_cross_section_Gaussian(wn_grid, v, alpha, coef)
xsec_l = exomol_cross_section_Lorentzian(wn_grid, v, gamma, coef)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
xsec_swv = exomol_cross_section_scipy_wofz_Voigt(wn_grid, v, alpha, gamma, coef)
xsec_wv = exomol_cross_section_width_Voigt(wn_grid, v, alpha, gamma, coef)
xsec_pv = exomol_cross_section_pseudo_Voigt(wn_grid, v, alpha, gamma, coef)        
xsec_pfv = exomol_cross_section_pseudo_FWHM_Voigt(wn_grid, v, alpha, gamma, coef)

In [ ]:
plot_xsec(wn_grid, xsec_g, database, 'Gaussian')
plot_xsec(wn_grid, xsec_l, database, 'Lorentzian')
plot_xsec(wn_grid, xsec_sv, database, 'SciPyVoigt')
plot_xsec(wn_grid, xsec_swv, database, 'SciPyWofzVoigt')
plot_xsec(wn_grid, xsec_wv, database, 'WidthVoigt')
plot_xsec(wn_grid, xsec_pv, database, 'PseudoVoigt')
plot_xsec(wn_grid, xsec_pfv, database, 'PseudoFWHMVoigt')

In [ ]:
wn = wn_grid
fig, ax = plt.subplots()
ax.plot(wn, xsec_g, label='Gaussian profile')
ax.plot(wn, xsec_l, label='Lorentzian profile')
ax.plot(wn, xsec_sv, label='SciPy Voigt profile')
ax.plot(wn, xsec_swv, label='SciPy Wofz Voigt profile')
ax.plot(wn, xsec_wv, label='Width Voigt profile')
ax.plot(wn, xsec_pv, label='Pseudo Voigt profile')
ax.plot(wn, xsec_pfv, label='Pseudo FWHM Voigt profile')
ax.set_xlabel('Wavenumber, cm$^{-1}$')
ax.set_ylabel('Cross-section, cm$^{2}$/molecule')
ax.set_title(abs_emi+' Cross-Section ExoMol at T = '+str(T)+' K') 
ax.legend() 
plots_foldername = saveplots+molecule+'/'
if os.path.exists(plots_foldername):
    pass
else:
    os.makedirs(plots_foldername, exist_ok=True)    
plt.savefig(plots_foldername+molecule+'__T'+str(T)+'__'+str(min_wn)+'-'+str(max_wn)+'cm-1__'
            +database+'__'+abs_emi+'.png', dpi=500)
plt.show()
print('Cross sections plot saved.')


## Plot At Different T With The Same Line Profile

In [ ]:
profile = 'SciPyVoigt'
wn = wn_grid
plt.figure(figsize=(12,8), dpi=500)
fig, ax = plt.subplots()

T = 800
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T = 800 K')

T = 600
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T = 600 K')

T = 400
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T = 400 K')

T = 200
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T = 200 K')

T = 100
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T = 100 K')

T = 50
coef = cal_abscoefs(v, gp, A, Epp, Q, abundance)
#coef = cal_emicoefs(v, gp, A, Ep, Q, abundance)
alpha = Doppler_HWHM (v, mass, T)
gamma = Lorentzian_HWHM (T, P, n_air, gamma_L)
xsec_sv = exomol_cross_section_scipy_Voigt(wn_grid, v, alpha, gamma, coef)
ax.plot(wn, xsec_sv, linewidth=1, label='T =   50 K')

ax.set_xlabel('Wavenumber, cm$^{-1}$')
ax.set_ylabel('Cross-section, cm$^{2}$/molecule')
ax.set_title(database + ' Database ' + molecule + ' ' + abs_emi +' Cross-Section With ' + profile)  
ax.legend() 
plots_foldername = saveplots+molecule+'/'
if os.path.exists(plots_foldername):
    pass
else:
    os.makedirs(plots_foldername, exist_ok=True)    
plt.savefig(plots_foldername+molecule+'__'+str(min_wn)+'-'+str(max_wn)+'cm-1__'
            +database+'__'+abs_emi+'__'+profile+'.png', dpi=500)
plt.show()
print('Cross sections plot saved.')
